In [6]:
import warnings
warnings.filterwarnings("ignore")

import argparse
import os
import sys
import json
from pathlib import Path
import numpy as np

import pandas as pd
import datetime

import torch
import torch.backends.cudnn as cudnn

# FILE = Path(__file__).resolve()
# ROOT = FILE.parents[0]  # YOLOv5 root directory
# if str(ROOT) not in sys.path:
#     sys.path.append(str(ROOT))  # add ROOT to PATH
# ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.common import DetectMultiBackend
from utils.datasets import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync

In [7]:

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32,
                              np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [8]:
class ChinkensDetcet:
    def __init__(self, weights = 'runs/train/exp7/weights/best.pt',
                 data = 'data/chicken_yang.yaml'):
        self.weights = weights
        self.device = select_device('')
        self.data = data
        self.augment = False
        self.conf_thres = 0.25,  # confidence threshold
        self.iou_thres = 0.45,  # NMS IOU threshold
        self.classes = None,  # filter by class: --class 0, or --class 0 2 3
        self.agnostic_nms = False,  # class-agnostic NMS
        self.max_det = 1000
        self.cvfont = cv2.FONT_HERSHEY_SIMPLEX

        self.model = self.model_load(self.weights, self.device, self.data)

    def model_load(self, weights, device, data):
        self.model = DetectMultiBackend(weights, device=device, dnn=False, data=data, fp16=False)
        self.stride, self.names, self.pt = self.model.stride, self.model.names, self.model.pt
        # print(self.pt)
        self.imgsz = check_img_size((704, 704), s=self.stride)
        self.bs = 1

        self.model.warmup(imgsz=(1 if self.pt else self.bs, 3, *self.imgsz))

        print("model load...")
        return self.model

    def ell_chicks_pos(self, x, y, A, B, x1, y1):
        r = ((x - x1) ** 2) / (A ** 2) + ((y - y1) ** 2) / (B ** 2)
        return True if r <= 1 else False

    def load_data(self, source):
        paths = str(source)
        dataset = LoadImages(paths, img_size=self.imgsz, stride=self.stride, auto=self.pt)
        # print(dataset.count())
        # vid_path, vid_writer = [None] * self.bs, [None] * self.bs
        # dataset.cap.set(3, 640)
        # dataset.cap.set(4, 480)
        FPS = dataset.cap.get(cv2.CAP_PROP_FPS)
        self.FPS = int(round(FPS, 0))
        self.frames = dataset.frames
        self.second = int(self.frames / FPS)
        self.dataset = dataset


    def detcet_img(self, time_list, save_path, view_img = True, save_img = True, dtype = 'image'):
        dataset = self.dataset
        FPS = self.FPS
        frames = self.frames
        second = self.second


        dt, seen = [0.0, 0.0, 0.0], []

        for path, im, im0s, vid_cap, s in dataset:
            frame = dataset.frame - 1
            

            if frame % FPS != 0:
                continue
            # print(frame)
            times_second = int(frame / FPS)
            if times_second not in time_list:
                continue
            
            # seen += 1 # second
            t1 = time_sync()
            im = torch.from_numpy(im).to(self.device)
            im = im.half() if self.model.fp16 else im.float()
            im /= 255  # 0 - 255 to 0.0 - 1.0
            if len(im.shape) == 3:
                im = im[None]  # expand for batch dim
            t2 = time_sync()
            dt[0] += t2 - t1
            pred = self.model(im, augment=self.augment)
            t3 = time_sync()
            dt[1] += t3 - t2
            pred = non_max_suppression(pred)
            for i, det in enumerate(pred):
                im0 = im0s.copy()
                p, frame = path, getattr(dataset, 'frame', 0)
                p = Path(p)  # to Path
                s += '%gx%g ' % im.shape[2:]  # print string
                imc = im0.copy()
                scaless = []
                chickens = []
                if len(det):
                    # Rescale boxes from img_size to im0 size
                    det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()
                    # Print results
                    for c in det[:, -1].unique():
                        n = (det[:, -1] == c).sum()  # detections per class
                        s += f"{n} {self.names[int(c)]}{'s' * (n > 1)}, "  # add to string
                    min_scales_conf = 0.5
                    for *xyxy, conf, cls in reversed(det):
                        xyxy = [int(x.cpu().numpy()) for x in xyxy]
                        name_l = self.names[int(cls)]
                        if name_l == 'scales' and conf > min_scales_conf:
                            min_scales_conf = conf
                            scaless = [conf, xyxy]
                        if name_l == 'chicken':
                            chickens.append((conf, xyxy))
                chickens_numbers = 0
                img_h, img_w, img_c = imc.shape
                line_w = 6
                text_w = 2

                scales_min_w = 0.05
                scales_min_h = 0.75

                scales_max_w = 0.2
                scales_max_h = 0.02

                if scaless:
                    s_conf = str(scaless[0].cpu().numpy() * 100)[:5] + '%'
                    s_x1, s_y1, s_x2, s_y2 = scaless[1]
                    s_w, s_h = s_x2 - s_x1, s_y2 - s_y1
                    if view_img:
                        cv2.rectangle(imc, (s_x1, s_y1), (s_x2, s_y2), (50, 50, 220), line_w)
                        cv2.putText(imc, 'scales ' + s_conf, (s_x1, s_y1 - 10), self.cvfont, text_w, (50, 50, 220), line_w)
                    if chickens:
                        min_w = int(s_w * scales_min_w)
                        min_h = int(s_h * scales_min_h)

                        max_w = int(s_w * scales_max_w)
                        max_h = int(s_h * scales_max_h)

                        minx, miny = s_x1 + min_w, s_y1 + min_h
                        maxx, maxy = s_x2 - max_w, s_y2 - max_h

                        x_c, y_c = minx + int((maxx - minx) / 2), miny + int((maxy - miny) / 2)
                        A = int((maxx - minx) / 2)
                        B = int((maxy - miny) / 2)

                        # cv2.rectangle(imc, (minx, miny), (maxx, maxy), (255, 125, 60), line_w)
                        if view_img:
                            cv2.ellipse(imc, (x_c, y_c), (A, B), 0, 0, 360, (255, 125, 60), line_w)

                        for conf, xyxy in chickens:
                            c_x_conter, c_y_conter = int(xyxy[0] + (xyxy[2] - xyxy[0]) / 2), int(
                                xyxy[1] + (xyxy[3] - xyxy[1]) / 2 * 1.2)
                            c_r = self.ell_chicks_pos(x_c, y_c, A, B, c_x_conter, c_y_conter)
                            if c_r:
                                chickens_numbers += 1
                                if view_img:
                                    cv2.rectangle(imc, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (20, 235, 20), line_w)
                                    cv2.circle(imc, (c_x_conter, c_y_conter), 10, (20, 255, 20), -1)
                            else:
                                if view_img:
                                    # cv2.rectangle(imc, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (255, 255, 255), line_w-3)
                                    cv2.circle(imc, (c_x_conter, c_y_conter), 5, (255, 255, 255), -1)
                if view_img:
                    cv2.putText(imc, "chickens : " + str(chickens_numbers), (int(img_h * 0.1), int(img_w * 0.1)), self.cvfont,
                                text_w, (20, 235, 20), line_w)
                seen.append(chickens_numbers)
                # Stream results
                # im0 = annotator.result()
                
                if view_img:
                    im0 = imc.copy()
                    # cv2.imshow(str(p), im0)
                    cv2.imshow(str(p), cv2.resize(im0, (int(im0.shape[1] / 3), int(im0.shape[0] / 3))))
                    if cv2.waitKey(1) == 27:  # 1 millisecond
                        exit()
                    # cv2.destroyAllWindows()
                if save_img:
                    im0 = imc.copy()
                    if dataset.mode == 'image':
                        cv2.imwrite(save_path, im0)
                # LOGGER.info(f'{s}Done. ({t3 - t2:.3f}s)')
                if dtype == 'image':
                    return chickens_numbers
            if len(seen) == len(time_list):
                return seen
        return seen

In [9]:
def f_read_emotion_csv(path, jeouls='all'):  # jeouls : 'j01', j02', j03', 'all'
    df = pd.read_csv(path, skipinitialspace=True)

    g, w = [], []
    for i in range(1, 61):
        g.append('g%02d' % i)
        w.append('w%02d' % i)

    # getTime을 1초씩 증가하여 60개 만듦
    df['g01'] = pd.to_datetime(df['getTime'], format='%Y-%m-%d %H:%M:%S')
    for i in range(2, 61):
        df['g%02d' % i] = df['g01'] + datetime.timedelta(seconds=(i - 1))

    if (jeouls == 'j01') or (jeouls == 'all'):
        dfs = df[df['jeoulID'] == 1]
        ar_g = np.array(dfs[g]).reshape(-1, 1)
        ar_w = np.array(dfs[w]).reshape(-1, 1)

        dfa = pd.DataFrame(ar_g, columns=['getTime'])
        dfa['j01'] = ar_w
        # print(dfa.shape)

    if (jeouls == 'j02') or (jeouls == 'all'):
        dfs = df[df['jeoulID'] == 2]
        ar_g = np.array(dfs[g]).reshape(-1, 1)
        ar_w = np.array(dfs[w]).reshape(-1, 1)

        dfb = pd.DataFrame(ar_g, columns=['getTime'])
        dfb['j02'] = ar_w
        # print(dfb.shape)

    if (jeouls == 'j03') or (jeouls == 'all'):
        dfs = df[df['jeoulID'] == 3]
        ar_g = np.array(dfs[g]).reshape(-1, 1)
        ar_w = np.array(dfs[w]).reshape(-1, 1)

        dfc = pd.DataFrame(ar_g, columns=['getTime'])
        dfc['j03'] = ar_w
        # print(dfc.shape)

    if jeouls == 'all':
        df2 = pd.merge(dfa, dfb, on='getTime', how='outer')
        df2 = pd.merge(df2, dfc, on='getTime', how='outer')
        return df2
    elif jeouls == 'j01':
        return dfa
    elif jeouls == 'j02':
        return dfb
    elif jeouls == 'j03':
        return dfc


def f_get_valid_weight(df, jeoul='all', min_value=10, max_value=2000):
    df2 = df.copy()

    mask = (df2[jeoul] >= min_value) & (df2[jeoul] < max_value)
    return df2[mask]

def read_csv_data(fname, jeoul='all'):
    df = f_read_emotion_csv(fname, jeoul)
    if jeoul=='all':
        df21 = f_get_valid_weight(df=df, jeoul='j01')
        df22 = f_get_valid_weight(df=df, jeoul='j02')
        df23 = f_get_valid_weight(df=df, jeoul='j03')

        df2 = pd.concat([df21, df22, df23], axis=0, ignore_index=True)
    else:
        df2 = f_get_valid_weight(df, jeoul)

    # print()
    return df2

In [10]:
import threading
from threading import Lock,Thread
import time

class MyThread(threading.Thread):
    def __init__(self, df_path, df_save_path, video_path, yolo_model):
        super(MyThread, self).__init__()
        self.df_path = df_path
        self.video_path = video_path
        self.yolo_model = yolo_model
        self.df_save_path = df_save_path


    def run(self):
        CD = self.yolo_model
        df_path = self.df_path
        video_path = self.video_path
        df_save_path = self.df_save_path

        check_num = 'chickens_numbers'


        df = read_csv_data(df_path, jeoul='j01')
        df.set_index('getTime', inplace=True)
        df[check_num] = np.nan
        # print(df)

        video_names = os.listdir(video_path)

        for v_n in video_names:
            v_p = video_path + '/' + v_n
            CD.load_data(v_p)

            time_second = v_p.split('.')[1][-14:]
            # print(v_p)
            star_time = pd.Period(time_second, freq='S')
            second = CD.second
            end_time = star_time + second

            time_list_s_e = pd.to_datetime([str(star_time), str(end_time)], format='%Y-%m-%d %H:%M:%S')
            df_temp = df[time_list_s_e[0]:time_list_s_e[1]]

            if len(df_temp) != 0:
                time_list = df_temp.index
                time_list_temp = time_list.to_period(freq='S')
                detcet_time = (time_list_temp - star_time)

                time_list_temp = [int(str(t).split(' * ')[0][1:]) if str(t) != '<Second>' else 1 for t in detcet_time]

                result = CD.detcet_img(time_list_temp, None, view_img=False, save_img=False, dtype='video')

                if len(result) == len(time_list):
                    df_temp.loc[time_list, check_num] = result
                else:
                    print('{0}: error'.format(v_n))
            else:
                print('{0}: no time data in video '.format(v_n))

            # df = df[df[check_num] != np.nan]
            df_temp = df_temp.dropna(subset=[check_num])
            if len(df_temp) > 0:
                df_temp.to_csv(df_save_path+video_names[0].split('.')[0]+'.csv', index=True)
                print(video_path+'is ok!')
            else:
                print(video_path+'is no data!')

In [11]:
CD = ChinkensDetcet()

f_path = './datasets/0723/'
f_name = '20211210093325_KF002101_sensorData.csv'

csv_save_path = './datasets/0723/csv_0723/'

v_path = './datasets/0723/ch1/'
v_name_time_day = os.listdir(v_path)

myThread = []

for v_name in v_name_time_day:
    thread = MyThread(f_path+f_name, csv_save_path, v_path+v_name, CD)
    thread.start()


YOLOv5 🚀 2022-5-8 torch 1.10.1+cu111 CUDA:0 (NVIDIA A100-PCIE-40GB, 40536MiB)

Fusing layers... 
Model summary: 444 layers, 86180143 parameters, 0 gradients, 204.0 GFLOPs


model load...


[h264 @ 0x7f2e040f5240] SEI type 5 size 2560 truncated at 1928


드림농장_CH1_257_1_1_20211210065735.mp4: no time data in video 
드림농장_CH1_257_1_1_20211210171151.mp4: no time data in video 
드림농장_CH1_257_1_1_20211210200613.mp4: no time data in video 
드림농장_CH1_257_1_1_20211210174834.mp4: no time data in video 
드림농장_CH1_257_1_1_20211210182516.mp4: no time data in video 
드림농장_CH1_257_1_1_20211210071550.mp4: no time data in video 


[h264 @ 0x7f2e04180540] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e0478bec0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e059e1cc0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e05a1d600] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04180540] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e0478bec0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e059e1cc0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e05a1d600] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04180540] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e0478bec0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e059e1cc0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e05a1d600] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04180540] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e0478bec0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e059e1cc0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e05a1d600] SEI type 5 size 2560 truncated 

...

드림농장_CH1_257_1_1_20211210044906.mp4: no time data in video 
드림농장_CH1_257_1_1_20211210073411.mp4: no time data in video 
드림농장_CH1_257_1_1_20211210100943.mp4: no time data in video 


[h264 @ 0x7f2e04741e80] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04db8cc0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e042a4dc0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e09889300] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04741e80] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04db8cc0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e042a4dc0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e09889300] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04741e80] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04db8cc0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e042a4dc0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e09889300] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04741e80] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04db8cc0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e042a4dc0] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e09889300] SEI type 5 size 2560 truncated 

드림농장_CH1_257_1_1_20211210145412.mp4: no time data in video 


[h264 @ 0x7f2e08b10b00] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04f7e900] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04785a80] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04dc9000] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e08b10b00] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04f7e900] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04785a80] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04dc9000] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e08b10b00] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04f7e900] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04785a80] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04dc9000] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e08b10b00] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04f7e900] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04785a80] SEI type 5 size 2560 truncated at 1928
[h264 @ 0x7f2e04dc9000] SEI type 5 size 2560 truncated 

./datasets/0723/ch1/20211210is ok!


In [25]:
# CD = ChinkensDetcet()

# f_path = './datasets/'
# f_name = '20211210093325_KF002101_sensorData.csv'
# df = read_csv_data(f_path+f_name, jeoul='j01')

# df.set_index('getTime', inplace=True)
# df['chickens_numbers'] = np.nan
# # print(df)
# # print(df.index.values.dtype)

# v_path = './datasets/'
# v_n = '드림농장_CH1_257_1_1_20211210092418.mp4'

# CD.load_data(v_path+v_n)

# time_second = v_n.split('.')[0][-14:]
# star_time = pd.Period(time_second, freq='S')
# second = CD.second
# end_time = star_time + second

# time_list_s_e = pd.to_datetime([str(star_time), str(end_time)], format='%Y-%m-%d %H:%M:%S')
# df_temp = df[time_list_s_e[0]:time_list_s_e[1]]
# if len(df_temp) != 0:
#     print(v_n + ': do it now! ')
#     time_list = df_temp.index
#     time_list_temp = time_list.to_period(freq='S')
#     # print(time_list_temp)
#     detcet_time = (time_list_temp - star_time)
#     time_list_temp = [int(str(t).split(' * ')[0][1:]) if str(t) != '<Second>' else 1 for t in detcet_time]
#     print(time_list_temp)
#     result = CD.detcet_img(time_list_temp, None, view_img=False, save_img=False, dtype='video')

#     print(len(time_list), time_list)
#     print(len(result), result)
#     if len(result) == len(time_list):
#         df.loc[time_list, 'chickens_numbers'] = result
#     else:
#         print('error')

YOLOv5 🚀 2022-5-8 torch 1.10.1+cu111 CUDA:0 (NVIDIA A100-PCIE-40GB, 40536MiB)

Fusing layers... 
Model summary: 444 layers, 86180143 parameters, 0 gradients, 204.0 GFLOPs


model load...
드림농장_CH1_257_1_1_20211210092418.mp4: do it now! 
[76, 77, 78]
0
25
50
75
100


[h264 @ 0x55c741d57480] SEI type 5 size 2560 truncated at 1928


125
150
175
200
225
250
275
300
325
350


[h264 @ 0x55c74515e580] SEI type 5 size 2560 truncated at 1928


...

1375
1400
1425
1450
1475
1500
1525
1550
1575
1600


[h264 @ 0x55c748dbb5c0] SEI type 5 size 2560 truncated at 1928


1625
1650
1675
1700
1725
1750
1775
1800
1825
1850


[h264 @ 0x55c744e6b100] SEI type 5 size 2560 truncated at 1928


1875
1900
1925
1950
3 DatetimeIndex(['2021-12-10 09:25:34', '2021-12-10 09:25:35',
               '2021-12-10 09:25:36'],
              dtype='datetime64[ns]', name='getTime', freq=None)
3 [0, 0, 0]
